In [1]:
import os 
import pandas as pd 
import re 
from model_V2 import Config
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
import pickle 

/Users/hugo/Desktop/Projects/conditionned_theatre_play_generation/.env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("openwebtext", num_proc=3)

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 8013769
    })
})

In [4]:
#Take only 10% of the dataset
#AttributeError: 'DatasetDict' object has no attribute 'train_test_split
dataset = dataset['train'].train_test_split(test_size=0.99)
dataset.pop('test')

Dataset({
    features: ['text'],
    num_rows: 7933632
})

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 80137
    })
})

In [6]:


split_dataset = dataset["train"].train_test_split(test_size=0.0005, seed=2357, shuffle=True)
split_dataset['val'] = split_dataset.pop('test') # rename the test split to val

#self.preprocessor = PreprocessData(config)
split = 'train'
if split == 'train':
    processed_corpus = split_dataset['train']
elif split == 'test':
    processed_corpus = split_dataset['val']


#self.processed_corpus = self. preprocessor.tokenize(self.processed_corpus['text'])
vocab_chars = set(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j','k','l','m','n','o','p','q','r','s','t','u','v','w','x', 'y', 'z', ' ', '!', '"', "'", '(', ')', ',', '-', '.', ':', ';', '?', '[', '\n', ']', '{', '}', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0'])

print(vocab_chars)
vocab_size = len(sorted(vocab_chars))
stoi = {char: i for i, char in enumerate(sorted(vocab_chars))}
itos = {i: char for i, char in enumerate(sorted(vocab_chars))}
print(vocab_size)
print(stoi)
print(itos)
print(3)

all_ids = []
def process(example) :
    text = example['text']
    text.replace('…', '...').replace('”', '"').replace('’', "'")
    
    
    ids = [stoi[char.lower()] for char in text if char.lower() in stoi]
    # note: I think eot should be prepended not appended... hmm. it's called "eot" though...
    out = {'ids': ids, 'len': len(ids)}
    #all_ids.append(ids)
    #ut = {'all_ids': all_ids}
    
    return out

# tokenize the dataset
tokenized = split_dataset.map(
    process,
    remove_columns=['text'],
    desc="tokenizing the splits",
    num_proc=8,
)

{'j', 'r', '(', '1', 'u', 'y', '3', '0', '"', '6', 't', '[', 'e', 'w', '?', 'k', ']', '2', 'b', 'o', '}', 'i', 'm', ',', 'l', ')', '8', '!', '9', 'd', 'g', ';', 'f', '\n', 'p', ' ', 'z', 'a', "'", ':', 'c', 's', 'q', '4', 'h', 'v', 'n', '7', '5', '{', '-', 'x', '.'}
53
{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '?': 22, '[': 23, ']': 24, 'a': 25, 'b': 26, 'c': 27, 'd': 28, 'e': 29, 'f': 30, 'g': 31, 'h': 32, 'i': 33, 'j': 34, 'k': 35, 'l': 36, 'm': 37, 'n': 38, 'o': 39, 'p': 40, 'q': 41, 'r': 42, 's': 43, 't': 44, 'u': 45, 'v': 46, 'w': 47, 'x': 48, 'y': 49, 'z': 50, '{': 51, '}': 52}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: ',', 8: '-', 9: '.', 10: '0', 11: '1', 12: '2', 13: '3', 14: '4', 15: '5', 16: '6', 17: '7', 18: '8', 19: '9', 20: ':', 21: ';', 22: '?', 23: '[', 24: ']', 25: 'a', 26: 'b', 27: 'c', 28: 'd', 29: 'e',

tokenizing the splits (num_proc=8): 100%|██████████| 41/41 [00:01<00:00, 22.35 examples/s]


In [19]:
from torch.utils.data import Dataset

class OWTProcessData():
    def __init__(self, ctx_len):
        self.ctx_len = ctx_len

    def remove_small_corpus(self, dataset):
        """Remove all sentences that are smaller than min_len"""
        for split in dataset:
            dataset[split] = dataset[split].filter(lambda x: x['len'] >= self.ctx_len+1, num_proc=8)
        
    def create_samples(self, split):
        samples = []
        with tqdm(total=len(split), desc="Creating samples") as pbar:
            for sample in split:
            
                for i in range(sample['len'] - self.ctx_len):
                    input_seq = sample['ids'][i:i + self.ctx_len]
                    target_seq = sample['ids'][i + 1:i + 1 + self.ctx_len]
                    samples.append((input_seq, target_seq))
                pbar.update(1)
        return samples


        return dataset
    
    

In [17]:
#Save the tokenized dataset
class OpenWebTextDataset(Dataset):
    def __init__(self, ctx_len, tokenized_data, split_str : str):

        self.data = tokenized_data
        self.processor = OWTProcessData(ctx_len)
        #self.data = self.processor.remove_small_corpus(self.data)
        self.dataset = self.processor.create_samples(self.data[split_str])
        #Save the dataset (it has)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        input, target = self.dataset[idx]
        return input, target 

In [20]:
dataset = OpenWebTextDataset(128, tokenized, 'train')

Creating samples:  17%|█▋        | 13513/80096 [34:15<33:29, 33.14it/s]    

In [ ]:
with open('text_generation_dataset.pkl', 'wb') as file:
    pickle.dump(dataset, file)